In [ ]:
# 코랩에서 mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab

data = pd.read_csv('/content/prep_data.csv')
data

,본문,답변
0,안녕하세요 정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다 부디 자세한 설명...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량인도청구소송 및 차량점유이전금지...
1,년전에 대포통장이로 인해 피해을 봐서 재판결과가 불기소이유통지을 받았거든요 근데 년...,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다
2,연고도 다르고 일면식도 없는 살위의 형을 일을 통해 알게 됐습니다 제가 주식투자를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...
3,이행권고결정판결 채권압류 및 추심명령 인용 채권추심시중은행 통장 압류 및 ...,별도의 구체적 상담이 필요하나 이행권고결정은 기판력의 제한이 없으므로 소멸시효완성 ...
4,천만원정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사중이었습니...,가해자가 위 사건으로 구속기소 되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...
...,...,...
18266,제가 작년 월달 쯤 승소 판결 받은 후에 압류 절차를 진행을 해야하는데 혼자 스스로...,통장 압류 방법에 관하여 질문하셨습니다 민사소송에서 승소를 하여 그 판결이 확정된 ...
18267,민사재판 패소후 현재 등본은 부모님댁으로 되어있습니다 나와서 따로 혼자 사는데 부모...,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다 민...
18268,추완항소이고 제가 피고인이예요 공시송달로 소송을 모르고 있다가 처음부터 끝까지 공시...,민사소송 항소이유를 잘못 기재한 것과 관련하여 질문하셨습니다 부동산 임대차에 있어 ...
18269,작년 여름 사기피해금액 약 만원를 당하여 형사고소를 하였고 월경 형사고소한 것이 피...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질무하셨습니다 민사소송에서 ...


In [15]:
def get_df_token(data):

    mecab = Mecab()
    q_pos = []

    for i in range(len(data)):
        q_pos.append(mecab.pos(data['본문'][i]))
    data['본문_토큰'] = q_pos

    pos = ["NNG","NNP",   # 명사
         "VV","VA"   # 동사
         ]

    data['본문_토큰'] = data['본문_토큰'].apply(lambda x: [t for (t, p) in x if p in pos])

    return data

def get_ques_token(question):
    mecab = Mecab()
    ques_token = mecab.pos(question)
    ques_token = ques_remove(ques_token)

    return ques_token

def ques_remove(question) :

    pos = ["NNG","NNP",   # 명사
           "VV","VA"   # 동사
           ]

    ques_token = [[t for (t, p) in question if p in pos]]

    return ques_token

def jaccard_similarity(data, question):

    union = set(data).union(set(question))
    intersection = set(data).intersection(set(question))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, question, num):
    
    # data: 데이터프레임, question: 입력한 텍스트(질문), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data['본문_토큰'].apply(lambda x: jaccard_similarity(x, question[0]))

    return data[['본문', '답변', '본문_토큰', 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, question):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data['본문_토큰'])
    tfidf_question = tfidf_vectorizer.transform(question)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_question).reshape(-1, )

    return data[['본문', '답변', '본문_토큰', 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def cos_high(data):
    if data['cosine_similarity'][0] >= 0.6:
        return data['답변'][0]

    else:
        return print('모델구현')


In [3]:
mecab = Mecab()
q_pos = []

for i in range(len(data)):
    q_pos.append(mecab.pos(data['본문'][i]))
data['본문_토큰'] = q_pos

In [4]:
data['본문_토큰']

0        [(안녕, NNG), (하, XSV), (세요, EP+EF), (정말, MAG), ...
1        [(년, NNG), (전, NNG), (에, JKB), (대포통장, NNP), (이...
2        [(연고, NNG), (도, JX), (다르, VA), (고, EC), (일면식, ...
3        [(이행, NNG), (권고, NNG), (결정, NNG), (판결, NNG), (...
4        [(천만, NR), (원, NNBC), (정도, NNG), (의, JKG), (피해...
                               ...                        
18266    [(제, NP), (가, JKS), (작년, NNG), (월, NNG), (달, N...
18267    [(민사재판, NNP), (패, NNG), (소후, NNG), (현재, MAG), ...
18268    [(추완, NNG), (항소, NNG), (이, VCP), (고, EC), (제, ...
18269    [(작년, NNG), (여름, NNG), (사기, NNG), (피해, NNG), (...
18270    [(경찰서, NNG), (에, JKB), (신고, NNG), (해서, XSV+EC)...
Name: 본문_토큰, Length: 18271, dtype: object

In [16]:
question = "최근 와이프와 이혼결정을 하게 되었습니다 자식은 아들하나 있는데 이미 성인이 된 이후라 양육권에 관한건 괜찮은거 같은데 합의이혼이면 재산분할의 경우 어떻게 될까요"

In [17]:
data = get_df_token(data)
data

,본문,답변,본문_토큰
0,안녕하세요 정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다 부디 자세한 설명...,차량의 경우 현재 점유자인 아버지를 피고로 하여 차량인도청구소송 및 차량점유이전금지...,"[안녕, 상황, 힘들, 질문, 설명, 감사, 아버지, 명의, 사업자, 내, 사업, ..."
1,년전에 대포통장이로 인해 피해을 봐서 재판결과가 불기소이유통지을 받았거든요 근데 년...,소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다,"[년, 전, 대포통장, 피해, 재판, 결과, 불기소, 이유, 통지, 받, 뒤, 법원..."
2,연고도 다르고 일면식도 없는 살위의 형을 일을 통해 알게 됐습니다 제가 주식투자를 ...,별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강제집행...,"[연고, 다르, 일면식, 없, 살, 위, 형, 일, 알, 주식, 투자, 주식, 투자..."
3,이행권고결정판결 채권압류 및 추심명령 인용 채권추심시중은행 통장 압류 및 ...,별도의 구체적 상담이 필요하나 이행권고결정은 기판력의 제한이 없으므로 소멸시효완성 ...,"[이행, 권고, 결정, 판결, 채권, 압류, 추심, 명령, 인용, 채권, 추심, 시..."
4,천만원정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사중이었습니...,가해자가 위 사건으로 구속기소 되었다면 가해자를 피고로 하여 민사소송 및 강제집행으...,"[정도, 피해, 있, 고소, 인지, 사건, 넘어가, 검찰, 수사, 와중, 피의자, ..."
...,...,...,...
18266,제가 작년 월달 쯤 승소 판결 받은 후에 압류 절차를 진행을 해야하는데 혼자 스스로...,통장 압류 방법에 관하여 질문하셨습니다 민사소송에서 승소를 하여 그 판결이 확정된 ...,"[작년, 월, 달, 승소, 판결, 받, 후, 압류, 절차, 진행, 하, 혼자, 하,..."
18267,민사재판 패소후 현재 등본은 부모님댁으로 되어있습니다 나와서 따로 혼자 사는데 부모...,민사소송 패소 후 부모님 집에 압류가 들어올 수 있는지와 관련하여 질문하셨습니다 민...,"[민사재판, 패, 소후, 등본, 부모, 댁, 되, 사, 부모, 댁, 압류, 있]"
18268,추완항소이고 제가 피고인이예요 공시송달로 소송을 모르고 있다가 처음부터 끝까지 공시...,민사소송 항소이유를 잘못 기재한 것과 관련하여 질문하셨습니다 부동산 임대차에 있어 ...,"[추완, 항소, 피고인, 공시송달, 소송, 모르, 처음, 끝, 공시송달, 판결, 원..."
18269,작년 여름 사기피해금액 약 만원를 당하여 형사고소를 하였고 월경 형사고소한 것이 피...,민사소송 소장에 소송비용 청구를 빠드린 경우와 관련하여 질무하셨습니다 민사소송에서 ...,"[작년, 여름, 사기, 피해, 금액, 당하, 형사, 고소, 하, 월경, 형사, 피,..."


In [18]:
ques_token = get_ques_token(question)
ques_token

[['최근',
  '와이프',
  '이혼',
  '결정',
  '하',
  '되',
  '자식',
  '아들',
  '성인',
  '이후',
  '양육권',
  '괜찮',
  '같',
  '합의',
  '이혼',
  '재산',
  '분할',
  '경우']]

In [19]:
result = jaccard_high(data, ques_token, 100)
result = cos_similarity(result, ques_token)

In [20]:
result

,본문,답변,본문_토큰,jaccard_similarity,cosine_similarity
4252,소송이혼중 합의이혼 했습니다 위자료재산분할은 서로간 청구 하지 않는다는 합의서도 제...,우선 결론부터 말씀드리자면 법원에서 조정이 이루어 졌다면 불복은 어렵습니다 다만...,"[소송, 이혼, 합의, 이혼, 위자료, 재산, 분할, 청구, 하, 합의, 제출, 신...",0.217391,0.350708
4190,협의이혼으로 재산분할할 때 반드시 재산과 채무를 함께 분할하나요 이혼할 때 재산분할...,우선 결론부터 말씀드리면 재산분할을 할 때에는 재산과 채무를 함께 계산을 해서 순...,"[협의, 이혼, 재산, 분할, 때, 재산, 채무, 분할, 이혼, 때, 재산, 분할,...",0.200000,0.347080
4179,협의이혼으로 재산분할할 때 반드시 재산과 채무를 함께 분할하나요 이혼할 때 재산분할...,우선 결론부터 말씀드리면 재산분할은 순재산을 대상으로 합니다 순재산은 적극재산에서...,"[협의, 이혼, 재산, 분할, 때, 재산, 채무, 분할, 이혼, 때, 재산, 분할,...",0.200000,0.347080
4279,협의이혼을 하려고 합니다 그런데 걱정이 있어요 협의이혼으로 재산분할할 때 반드시 재...,우선 결론부터 말씀드리자면 재산분할을 할 때에는 재산과 채무를 함께 계산을 해서 ...,"[협의, 이혼, 하, 걱정, 있, 협의, 이혼, 재산, 분할, 때, 재산, 채무, ...",0.200000,0.339167
4340,재산을 조금 가져오더라도 양육권을 남편에게 넘기고 싶습니다 협의이혼하다 잘 안되면 ...,우선 결론부터 말씀드리면 배우자와 합의가 안 되면 자동으로 이혼소송으로 넘어가는 ...,"[재산, 가져오, 양육권, 남편, 넘기, 협의, 이혼, 되, 이혼, 자동, 이혼, ...",0.166667,0.308927
...,...,...,...,...,...
11868,대여금 승소한 다음 채권압류 및 추심명령을 신청했습니다 그 이후에 재산명시도 진행하...,민사소송 승소 후 강제집행절차 진행방법에 관해서는 아래 글을 참고하시면 도움이 되...,"[대여금, 승소, 다음, 채권, 압류, 추심, 명령, 신청, 이후, 재산, 명시도,...",0.125000,0.042405
461,채무불이행명부 등록 기각 문의 통상적으로 질문 드립니다 채무불이행명부 등록을 하려고...,관련 법조 민사집행법 제조채무자의 감치 및 벌칙 ①채무자가 정당한 사유 없이 다음...,"[채무불이행, 명부, 등록, 기각, 문, 통상, 질문, 채무불이행, 명부, 등록, ...",0.138889,0.042333
10682,저는 피고이고 심 결과에 불복해 항소를 하였습니다 항소이유서는 심 때 변론을 맡았던...,항소이유서에 대해 상대측에서 답변서를 제출할 것인데 그 답변서에 심에서 등장하지 않...,"[피고, 심, 결과, 불복, 항소, 하, 항소, 이유, 심, 때, 변론, 맡, 변호...",0.125000,0.041044
5621,채권자가 여러 명이 있는데 일부에게 주소불명을 이유로 송달이 되지 않은 경우에는 어...,우선 결론부터 말씀드리면 공시송달로 진행할 수 있고 채권자에게 도달된 날로부터 주...,"[채권자, 있, 일부, 주소, 불명, 이유, 송달, 되, 경우, 하]",0.125000,0.040186
